In [1]:
# Dependencies
import json
import requests
import pandas as pd
from scipy import stats
from api_keys import user_agent

## Basic Setting up for U.S. Securities and Exchange Commission(SEC)
- Searching CIK# for Tesla and Ford Motor

In [2]:
# create request header
headers = {'User-Agent': user_agent}

In [3]:
# get all companies ticker data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

In [4]:
# dictionary to dataframe
companyTicker_df = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')
companyTicker_df.head()

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1045810,NVDA,NVIDIA CORP


In [5]:
# add leading zeros to CIK#
companyTicker_df['cik_str'] = companyTicker_df['cik_str'].astype(
                           str).str.zfill(10)

In [6]:
# review data
companyTicker_df.head()

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001045810,NVDA,NVIDIA CORP


In [7]:
# To get the CIK# of Tesla and Ford Motor by using their ticker#
cik_tesla = companyTicker_df.loc[companyTicker_df["ticker"]=="TSLA"].cik_str[0]
cik_ford = companyTicker_df.loc[companyTicker_df["ticker"]=="F"].cik_str[0]

In [8]:
# Print out tesla CIK#
cik_tesla

'0001318605'

In [9]:
# Print out ford motor CIK#
cik_ford

'0000037996'

## Ford Financial Data

In [10]:
# get company facts data (Ford)
companyFacts_ford = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_ford}.json',
    headers=headers
    )

In [136]:
# To get the Ford's financial statement account subject line items
companyFacts_ford.json()['facts']['us-gaap'].keys()

dict_keys(['AccountsPayableAndAccruedLiabilitiesCurrent', 'AccountsPayableCurrentAndNoncurrent', 'AccountsPayableRelatedPartiesCurrentAndNoncurrent', 'AccountsReceivableNetCurrent', 'AccountsReceivableRelatedParties', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdditionalPaidInCapitalCommonStock', 'AdditionsToNoncurrentAssets', 'AdjustmentForAmortization', 'AdvertisingExpense', 'AllocatedShareBasedCompensationExpenseNetOfTax', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AllowanceForNotesAndLoansReceivableCurrent', 'AllowanceForNotesAndLoansReceivableNoncurrent', 'Amortizatio

In [13]:
# To get the historical data of Ford's cash and cash equivalents and restricted cash(end of period) 
# and transfer into DataFrame and drop any rows with missing data
cashAndCashEquivaAtTheEnd_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cashAndCashEquivaAtTheEnd_ford_df = cashAndCashEquivaAtTheEnd_ford_df[-14:].reset_index(drop=True).rename(columns={'val':'netCashFlowEndOfPeriod'})
cashAndCashEquivaAtTheEnd_ford_df

,end,netCashFlowEndOfPeriod,accn,fy,fp,form,filed,frame
0,2020-03-31,26160000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,CY2020Q1I
1,2020-06-30,31181000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,CY2020Q2I
2,2020-09-30,24472000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,CY2020Q3I
3,2020-12-31,25935000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2020Q4I
4,2021-03-31,22050000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,CY2021Q1I
5,2021-06-30,23162000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2021Q2I
6,2021-09-30,27624000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2021Q3I
7,2021-12-31,20737000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2021Q4I
8,2022-03-31,21283000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2022Q1I
9,2022-06-30,20134000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q2I


In [14]:
# Prepare a new ford dataframe for collecting all financial information
ford_financial_df = cashAndCashEquivaAtTheEnd_ford_df[['end','netCashFlowEndOfPeriod']].copy()

# Add a new column for company name
ford_financial_df['Company_Name'] = "Ford Motor"

# Reorder the dataframe
ford_financial_df = ford_financial_df[['end','Company_Name','netCashFlowEndOfPeriod']]

# Review the dataframe
ford_financial_df

,end,Company_Name,netCashFlowEndOfPeriod
0,2020-03-31,Ford Motor,26160000000
1,2020-06-30,Ford Motor,31181000000
2,2020-09-30,Ford Motor,24472000000
3,2020-12-31,Ford Motor,25935000000
4,2021-03-31,Ford Motor,22050000000
5,2021-06-30,Ford Motor,23162000000
6,2021-09-30,Ford Motor,27624000000
7,2021-12-31,Ford Motor,20737000000
8,2022-03-31,Ford Motor,21283000000
9,2022-06-30,Ford Motor,20134000000


In [15]:
# To get the historical data of Ford's cash flow from operating and transfer into DataFrame
# drop any duplicated rows by column "end" (i.e.the end of reporting period)
cfo_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['NetCashProvidedByUsedInOperatingActivities']['units']['USD'])).\
               drop_duplicates(subset=['end'])
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cfo_ford_df = cfo_ford_df[-14:].reset_index(drop=True)
cfo_ford_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,-473000000,0000037996-20-000041,2020,Q1,10-Q,2020-04-29,NaN
1,2020-01-01,2020-06-30,8642000000,0000037996-20-000057,2020,Q2,10-Q,2020-07-31,NaN
2,2020-01-01,2020-09-30,19730000000,0000037996-20-000071,2020,Q3,10-Q,2020-10-29,NaN
3,2020-01-01,2020-12-31,24269000000,0000037996-21-000012,2020,FY,10-K,2021-02-05,NaN
4,2021-01-01,2021-03-31,4492000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,NaN
5,2021-01-01,2021-06-30,5248000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,NaN
6,2021-01-01,2021-09-30,12256000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,NaN
7,2021-01-01,2021-12-31,15787000000,0000037996-22-000013,2021,FY,10-K,2022-02-04,NaN
8,2022-01-01,2022-03-31,-1084000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,NaN
9,2022-01-01,2022-06-30,1863000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,NaN


In [16]:
# Adjust some rows data to reflect quarterly data

# Create an empty list to store the differences (2021-2022)
new_val = []

# Calculate the differences between each row
for i in range(12):
    if i % 4 == 0:
        new_val.append(cfo_ford_df.at[i, 'val']) # Keep the Q1 data as same
    else:
        difference = cfo_ford_df.at[i, 'val'] - cfo_ford_df.at[i - 1, 'val']
        new_val.append(difference)
        
# Add the 2023 Q1 and Q2 data to the new list
new_val.append(cfo_ford_df.at[12, 'val'])
new_val.append(cfo_ford_df.at[13, 'val']-cfo_ford_df.at[12, 'val'])

In [17]:
# Add a new column to ford financial dataframe
ford_financial_df['cfo'] = new_val
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo
9,2022-06-30,Ford Motor,20134000000,2947000000
10,2022-09-30,Ford Motor,21721000000,3812000000
11,2022-12-31,Ford Motor,25340000000,1178000000
12,2023-03-31,Ford Motor,22359000000,2800000000
13,2023-06-30,Ford Motor,26623000000,5035000000


In [63]:
# To get the historical data of Ford's equity
# and transfer into DataFrame and drop any rows with missing data
totalSHequity_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['StockholdersEquity']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
totalSHequity_ford_df = totalSHequity_ford_df[-14:].reset_index(drop=True).rename(columns={'val':'totalShareholderEquity'})
totalSHequity_ford_df.tail()

,end,totalShareholderEquity,accn,fy,fp,form,filed,frame
9,2022-06-30,44169000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,42125000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,43242000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,42366000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q1I
13,2023-06-30,43677000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [65]:
# Add a new column to tesla financial dataframe
ford_financial_df['totalShareholderEquity'] = totalSHequity_ford_df['totalShareholderEquity']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000


In [81]:
# To get the historical data of Ford's accounts payable
# and transfer into DataFrame and drop any rows with missing data
acctpay_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['AccountsPayableAndAccruedLiabilitiesCurrent']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
acctpay_ford_df = acctpay_ford_df[-14:].reset_index(drop=True)
acctpay_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,23378000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,27051000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,25605000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,26028000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,27749000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [82]:
# To get the historical data of Ford's current other liabilities
# and transfer into DataFrame and drop any rows with missing data
currentOtherLiabilities_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['OtherLiabilitiesCurrent']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentOtherLiabilities_ford_df = currentOtherLiabilities_ford_df[-14:].reset_index(drop=True)
currentOtherLiabilities_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,18895000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,19066000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,21097000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,21827000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,23925000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [83]:
# To get the historical data of Ford's current liabilities
# and transfer into DataFrame and drop any rows with missing data
currentLiabilities_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentLiabilities_ford_df = currentLiabilities_ford_df[-14:].reset_index(drop=True)
currentLiabilities_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,86452000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,90167000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,96866000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,95905000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,101015000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [85]:
# Calculate the total short-term debt(Ford)
total_short_term_debt_ford = currentLiabilities_ford_df['val'] - currentOtherLiabilities_ford_df['val'] - acctpay_ford_df['val']
total_short_term_debt_ford

# Add a new column to ford financial dataframe
ford_financial_df['totalShortTermDebt'] = total_short_term_debt_ford
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000


In [88]:
# To get the historical data of Ford's non-current other liabilities
# and transfer into DataFrame and drop any rows with missing data
noncurrentOtherLiabilities_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['OtherLiabilitiesNoncurrent']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
noncurrentOtherLiabilities_ford_df = noncurrentOtherLiabilities_ford_df[-14:].reset_index(drop=True)
noncurrentOtherLiabilities_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,28221000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,28591000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,25497000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,25908000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,25754000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [93]:
# To get the historical data of Ford's total liabilities
# and transfer into DataFrame and drop any rows with missing data
totalLiabilities_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['Liabilities']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
totalLiabilities_ford_df = totalLiabilities_ford_df[-14:].reset_index(drop=True)
totalLiabilities_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,201518000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,204830000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,212717000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,214603000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,222385000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [105]:
# To get the historical data of Ford's deferred tax income liabilities 
# and transfer into DataFrame and drop any rows with missing data
dtl_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['DeferredTaxLiabilitiesOther']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
dtl_ford_df = dtl_ford_df[-14:].reset_index(drop=True)
dtl_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,1837000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,1793000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,1549000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,1548000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,1721000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [109]:
# Calculate the total long-term debt(Ford)
total_long_term_debt_ford = totalLiabilities_ford_df['val'] - dtl_ford_df['val'] - noncurrentOtherLiabilities_ford_df['val'] - currentLiabilities_ford_df['val']
total_long_term_debt_ford

# Add a new column to ford financial dataframe
ford_financial_df['totalLongTermDebt'] = total_long_term_debt_ford
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000


In [112]:
# To get the historical data of Ford's cash and cash equivalents
# and transfer into DataFrame and drop any rows with missing data
cash_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['CashAndCashEquivalentsAtCarryingValue']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cash_ford_df = cash_ford_df[-14:].reset_index(drop=True).rename(columns={'val':'cashAndCashEquiva'})
cash_ford_df

,end,cashAndCashEquiva,accn,fy,fp,form,filed,frame
0,2020-03-31,25971000000,0000037996-20-000041,2020,Q1,10-Q,2020-04-29,CY2020Q1I
1,2020-06-30,30989000000,0000037996-20-000057,2020,Q2,10-Q,2020-07-31,CY2020Q2I
2,2020-09-30,24263000000,0000037996-20-000071,2020,Q3,10-Q,2020-10-29,CY2020Q3I
3,2020-12-31,25243000000,0000037996-22-000013,2021,FY,10-K,2022-02-04,CY2020Q4I
4,2021-03-31,21826000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,CY2021Q1I
5,2021-06-30,22955000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,CY2021Q2I
6,2021-09-30,27429000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,CY2021Q3I
7,2021-12-31,20540000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2021Q4I
8,2022-03-31,21013000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,CY2022Q1I
9,2022-06-30,19516000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I


In [114]:
# Add a new column to ford financial dataframe
ford_financial_df['cashAndCashEquiva'] = cash_ford_df['cashAndCashEquiva']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000


In [115]:
# Add a new column to ford financial dataframe(current liabilities)
ford_financial_df['CL'] = currentLiabilities_ford_df['val']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000


In [117]:
# To get the historical data of Ford's current assets
# and transfer into DataFrame and drop any rows with missing data
currentassets_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentassets_ford_df = currentassets_ford_df[-14:].reset_index(drop=True).rename(columns={'val':'CA'})
currentassets_ford_df.tail()

,end,CA,accn,fy,fp,form,filed,frame
9,2022-06-30,100469000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,108088000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,116476000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,115123000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,121712000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [121]:
# Add a new column to ford financial dataframe
ford_financial_df['CA'] = currentassets_ford_df['CA']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000


In [125]:
# To get the historical data of Ford's inventory
# and transfer into DataFrame and drop any rows with missing data
inventory_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['InventoryNet']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
inventory_ford_df = inventory_ford_df[-14:].reset_index(drop=True)
inventory_ford_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,13976000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2022Q2I
10,2022-09-30,15213000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3I
11,2022-12-31,14080000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q4I
12,2023-03-31,16212000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1I
13,2023-06-30,17703000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2I


In [126]:
# Add a new column to ford financial dataframe
ford_financial_df['inventory'] = inventory_ford_df['val']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000,13976000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000,15213000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000,14080000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000,16212000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000,17703000000


In [129]:
# To get the historical data of Ford's Income from Operations
# and transfer into DataFrame and drop any rows with missing data
OperatingIncome_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['OperatingIncomeLoss']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
OperatingIncome_ford_df = OperatingIncome_ford_df[-14:].reset_index(drop=True)
OperatingIncome_ford_df.tail()

,start,end,val,accn,fy,fp,form,filed,frame
9,2022-04-01,2022-06-30,2868000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q2
10,2022-07-01,2022-09-30,504000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2022Q3
11,2022-01-01,2022-12-31,6276000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2022
12,2023-01-01,2023-03-31,2113000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2023Q1
13,2023-04-01,2023-06-30,2461000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2023Q2


In [131]:
# Add a new column to ford financial dataframe
ford_financial_df['incomeFromOperations'] = OperatingIncome_ford_df['val']
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000,13976000000,2868000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000,15213000000,504000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000,14080000000,6276000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000,16212000000,2113000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000,17703000000,2461000000


In [141]:
# To get the historical data of Ford's total revenues
# and transfer into DataFrame and drop any rows with missing data
totalRevenue_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['RevenueFromContractWithCustomerExcludingAssessedTax']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
totalRevenue_ford_df = totalRevenue_ford_df[-14:].reset_index(drop=True)
totalRevenue_ford_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,34320000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,CY2020Q1
1,2020-04-01,2020-06-30,19371000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,CY2020Q2
2,2020-07-01,2020-09-30,37501000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,CY2020Q3
3,2020-01-01,2020-12-31,127144000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2020
4,2021-01-01,2021-03-31,36228000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,CY2021Q1
5,2021-04-01,2021-06-30,26752000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2021Q2
6,2021-07-01,2021-09-30,35683000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2021Q3
7,2021-01-01,2021-12-31,136341000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2021
8,2022-01-01,2022-03-31,34476000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2022Q1
9,2022-04-01,2022-06-30,40190000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q2


In [143]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the differences
new_total_rev_ford = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = totalRevenue_ford_df.at[i, 'val'] - totalRevenue_ford_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = totalRevenue_ford_df.at[i, 'val']
    new_total_rev_ford.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
new_total_rev_ford.append(totalRevenue_ford_df.at[12, 'val'])
new_total_rev_ford.append(totalRevenue_ford_df.at[13, 'val'])

In [146]:
# Add a new column to ford financial dataframe
ford_financial_df['totalRevenue'] = new_total_rev_ford
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000,13976000000,2868000000,40190000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000,15213000000,504000000,39392000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000,14080000000,6276000000,43999000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000,16212000000,2113000000,41474000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000,17703000000,2461000000,44954000000


In [152]:
# To get the historical data of Ford's net Income Attributable to common stockholders and transfer into DataFrame and drop any rows with missing data
# Drop any duplicated rows by column "end" (i.e. the end of reporting period)
netIncomeAttrCommonStock_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['NetIncomeLossAvailableToCommonStockholdersBasic']['units']['USD'])).\
               dropna().drop_duplicates(subset=['end'])
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
netIncomeAttrCommonStock_ford_df = netIncomeAttrCommonStock_ford_df[-14:].reset_index(drop=True)
netIncomeAttrCommonStock_ford_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,-1993000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,CY2020Q1
1,2020-04-01,2020-06-30,1117000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,CY2020Q2
2,2020-07-01,2020-09-30,2385000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,CY2020Q3
3,2020-01-01,2020-12-31,-1279000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2020
4,2021-01-01,2021-03-31,3262000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,CY2021Q1
5,2021-04-01,2021-06-30,561000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2021Q2
6,2021-07-01,2021-09-30,1832000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2021Q3
7,2021-01-01,2021-12-31,17937000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2021
8,2022-01-01,2022-03-31,-3110000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2022Q1
9,2022-04-01,2022-06-30,667000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q2


In [153]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the new quarterly data
netIncomeAttrCommonStock = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = netIncomeAttrCommonStock_ford_df.at[i, 'val'] - netIncomeAttrCommonStock_ford_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = netIncomeAttrCommonStock_ford_df.at[i, 'val']
    netIncomeAttrCommonStock.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
netIncomeAttrCommonStock.append(netIncomeAttrCommonStock_ford_df.at[12, 'val'])
netIncomeAttrCommonStock.append(netIncomeAttrCommonStock_ford_df.at[13, 'val'])

In [155]:
# Add a new column to ford financial dataframe
ford_financial_df['netIncomeAttrCommonStock'] = netIncomeAttrCommonStock
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue,netIncomeAttrCommonStock
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000,13976000000,2868000000,40190000000,667000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000,15213000000,504000000,39392000000,-827000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000,14080000000,6276000000,43999000000,1289000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000,16212000000,2113000000,41474000000,1757000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000,17703000000,2461000000,44954000000,1917000000


In [160]:
# To get the historical data of Ford's cost of sales
# and transfer into DataFrame and drop any rows with missing data
cogs_ford_df = pd.DataFrame.from_dict((
               companyFacts_ford.json()['facts']['us-gaap']['CostOfGoodsAndServicesSold']['units']['USD'])).\
               dropna()
# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cogs_ford_df = cogs_ford_df[-14:].reset_index(drop=True)
cogs_ford_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,30522000000,0000037996-21-000026,2021,Q1,10-Q,2021-04-29,CY2020Q1
1,2020-04-01,2020-06-30,17932000000,0000037996-21-000058,2021,Q2,10-Q,2021-07-29,CY2020Q2
2,2020-07-01,2020-09-30,31223000000,0000037996-21-000086,2021,Q3,10-Q,2021-10-28,CY2020Q3
3,2020-01-01,2020-12-31,112752000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2020
4,2021-01-01,2021-03-31,29297000000,0000037996-22-000024,2022,Q1,10-Q,2022-04-28,CY2021Q1
5,2021-04-01,2021-06-30,22904000000,0000037996-22-000048,2022,Q2,10-Q,2022-07-28,CY2021Q2
6,2021-07-01,2021-09-30,30057000000,0000037996-22-000073,2022,Q3,10-Q,2022-10-27,CY2021Q3
7,2021-01-01,2021-12-31,114651000000,0000037996-23-000012,2022,FY,10-K,2023-02-03,CY2021
8,2022-01-01,2022-03-31,29036000000,0000037996-23-000029,2023,Q1,10-Q,2023-05-03,CY2022Q1
9,2022-04-01,2022-06-30,33191000000,0000037996-23-000050,2023,Q2,10-Q,2023-07-28,CY2022Q2


In [162]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the new quarterly data
new_cogs_ford = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = cogs_ford_df.at[i, 'val'] - cogs_ford_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = cogs_ford_df.at[i, 'val']
    new_cogs_ford.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
new_cogs_ford.append(cogs_ford_df.at[12, 'val'])
new_cogs_ford.append(cogs_ford_df.at[13, 'val'])

print(new_cogs_ford)

[30522000000, 17932000000, 31223000000, 33075000000, 29297000000, 22904000000, 30057000000, 32393000000, 29036000000, 33191000000, 34354000000, 37816000000, 34669000000, 37471000000]


In [164]:
# Calculate the gross profit(Ford)
gross_profit_ford = ford_financial_df['totalRevenue'] - new_cogs_ford

# Add a new column to ford financial dataframe
ford_financial_df['grossProfit'] = gross_profit_ford
ford_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue,netIncomeAttrCommonStock,grossProfit
9,2022-06-30,Ford Motor,20134000000,2947000000,44169000000,44179000000,85008000000,19516000000,86452000000,100469000000,13976000000,2868000000,40190000000,667000000,6999000000
10,2022-09-30,Ford Motor,21721000000,3812000000,42125000000,44050000000,84279000000,21548000000,90167000000,108088000000,15213000000,504000000,39392000000,-827000000,5038000000
11,2022-12-31,Ford Motor,25340000000,1178000000,43242000000,50164000000,88805000000,25134000000,96866000000,116476000000,14080000000,6276000000,43999000000,1289000000,6183000000
12,2023-03-31,Ford Motor,22359000000,2800000000,42366000000,48050000000,91242000000,22144000000,95905000000,115123000000,16212000000,2113000000,41474000000,1757000000,6805000000
13,2023-06-30,Ford Motor,26623000000,5035000000,43677000000,49341000000,93895000000,26406000000,101015000000,121712000000,17703000000,2461000000,44954000000,1917000000,7483000000


In [165]:
# Write the DataFrame to a excel file
ford_financial_df.to_excel("Outputs/ford_financial_measures.xlsx", index=False)

## Tesla Financial Data

In [18]:
# get company facts data (Tesla)
companyFacts_tesla = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_tesla}.json',
    headers=headers
    )

In [19]:
# To get the Tesla's financial statement account subject line items
companyFacts_tesla.json()['facts']['us-gaap'].keys()

dict_keys(['AccountsAndNotesReceivableNet', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccretionAmortizationOfDiscountsAndPremiumsInvestments', 'AccrualForEnvironmentalLossContingencies', 'AccrualForEnvironmentalLossContingenciesPayments', 'AccruedEnvironmentalLossContingenciesCurrent', 'AccruedEnvironmentalLossContingenciesNoncurrent', 'AccruedLiabilitiesCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdditionalPaidInCapitalCommonStock', 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities', 'AdjustmentsToAdditionalPaidInCapitalEquityComponentOfConvertibleDebt', 'AdjustmentsToAdditionalPaidInCapitalOther', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdjustmentsToAdditionalPaidInCapitalWar

In [20]:
# To get the historical data of Tesla's cash and cash equivalents and restricted cash(end of period) 
# and transfer into DataFrame and drop any rows with missing data
cashAndCashEquivaAtTheEnd_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations']['units']['USD'])).\
               dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cashAndCashEquivaAtTheEnd_tesla_df = cashAndCashEquivaAtTheEnd_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'netCashFlowEndOfPeriod'})

In [21]:
# Prepare a new telsa dataframe for collecting all financial information
tsla_financial_df = cashAndCashEquivaAtTheEnd_tesla_df[['end','netCashFlowEndOfPeriod']].copy()

# Add a new column for company name
tsla_financial_df['Company_Name'] = "Tesla"

# Reorder the dataframe
tsla_financial_df = tsla_financial_df[['end','Company_Name','netCashFlowEndOfPeriod']]

# Review the dataframe
tsla_financial_df

,end,Company_Name,netCashFlowEndOfPeriod
0,2020-03-31,Tesla,8547000000
1,2020-06-30,Tesla,9106000000
2,2020-09-30,Tesla,15003000000
3,2020-12-31,Tesla,19901000000
4,2021-03-31,Tesla,17723000000
5,2021-06-30,Tesla,16825000000
6,2021-09-30,Tesla,16694000000
7,2021-12-31,Tesla,18144000000
8,2022-03-31,Tesla,18040000000
9,2022-06-30,Tesla,18887000000


In [22]:
# To get the historical data of Tesla's cash flow from operating and transfer into DataFrame
# Drop duplicated rows per column reporting end date "end"
cfo_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['NetCashProvidedByUsedInOperatingActivities']['units']['USD'])).drop_duplicates(subset=['end'])

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cfo_tesla_df = cfo_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'cfo'})
cfo_tesla_df

,start,end,cfo,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,-440000000,0001564590-20-019931,2020,Q1,10-Q,2020-04-30,NaN
1,2020-01-01,2020-06-30,524000000,0001564590-20-033670,2020,Q2,10-Q,2020-07-28,NaN
2,2020-01-01,2020-09-30,2924000000,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,NaN
3,2020-01-01,2020-12-31,5943000000,0001564590-21-004599,2020,FY,10-K,2021-02-08,NaN
4,2021-01-01,2021-03-31,1641000000,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,NaN
5,2021-01-01,2021-06-30,3765000000,0000950170-21-000524,2021,Q2,10-Q,2021-07-27,NaN
6,2021-01-01,2021-09-30,6912000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,NaN
7,2021-01-01,2021-12-31,11497000000,0000950170-22-000796,2021,FY,10-K,2022-02-07,NaN
8,2022-01-01,2022-03-31,3995000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,NaN
9,2022-01-01,2022-06-30,6346000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,NaN


In [23]:
# Adjust some rows data to reflect quarterly data

# 2023 Q2 Data
Q2_2023_cfo = cfo_tesla_df.iloc[13]['cfo'] - cfo_tesla_df.iloc[12]['cfo']
cfo_tesla_df.at[13, "cfo"] = Q2_2023_cfo

# 2022 Q2,Q3,Q4 Data
Q4_2022_cfo = cfo_tesla_df.iloc[11]['cfo'] - cfo_tesla_df.iloc[10]['cfo']
Q3_2022_cfo = cfo_tesla_df.iloc[10]['cfo'] - cfo_tesla_df.iloc[9]['cfo']
Q2_2022_cfo = cfo_tesla_df.iloc[9]['cfo'] - cfo_tesla_df.iloc[8]['cfo']
cfo_tesla_df.at[11, "cfo"] = Q4_2022_cfo
cfo_tesla_df.at[10, "cfo"] = Q3_2022_cfo
cfo_tesla_df.at[9, "cfo"] = Q2_2022_cfo

# 2021 Q2,Q3,Q4 Data
Q4_2021_cfo = cfo_tesla_df.iloc[7]['cfo'] - cfo_tesla_df.iloc[6]['cfo']
Q3_2021_cfo = cfo_tesla_df.iloc[6]['cfo'] - cfo_tesla_df.iloc[5]['cfo']
Q2_2021_cfo = cfo_tesla_df.iloc[5]['cfo'] - cfo_tesla_df.iloc[4]['cfo']
cfo_tesla_df.at[7, "cfo"] = Q4_2021_cfo
cfo_tesla_df.at[6, "cfo"] = Q3_2021_cfo
cfo_tesla_df.at[5, "cfo"] = Q2_2021_cfo

# 2020 Q2,Q3,Q4 Data
Q4_2020_cfo = cfo_tesla_df.iloc[3]['cfo'] - cfo_tesla_df.iloc[2]['cfo']
Q3_2020_cfo = cfo_tesla_df.iloc[2]['cfo'] - cfo_tesla_df.iloc[1]['cfo']
Q2_2020_cfo = cfo_tesla_df.iloc[1]['cfo'] - cfo_tesla_df.iloc[0]['cfo']
cfo_tesla_df.at[3, "cfo"] = Q4_2020_cfo
cfo_tesla_df.at[2, "cfo"] = Q3_2020_cfo
cfo_tesla_df.at[1, "cfo"] = Q2_2020_cfo

cfo_tesla_df

,start,end,cfo,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,-440000000,0001564590-20-019931,2020,Q1,10-Q,2020-04-30,NaN
1,2020-01-01,2020-06-30,964000000,0001564590-20-033670,2020,Q2,10-Q,2020-07-28,NaN
2,2020-01-01,2020-09-30,2400000000,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,NaN
3,2020-01-01,2020-12-31,3019000000,0001564590-21-004599,2020,FY,10-K,2021-02-08,NaN
4,2021-01-01,2021-03-31,1641000000,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,NaN
5,2021-01-01,2021-06-30,2124000000,0000950170-21-000524,2021,Q2,10-Q,2021-07-27,NaN
6,2021-01-01,2021-09-30,3147000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,NaN
7,2021-01-01,2021-12-31,4585000000,0000950170-22-000796,2021,FY,10-K,2022-02-07,NaN
8,2022-01-01,2022-03-31,3995000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,NaN
9,2022-01-01,2022-06-30,2351000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,NaN


In [24]:
# Add a new column to tesla financial dataframe
tsla_financial_df['cfo'] = cfo_tesla_df['cfo']
tsla_financial_df.head()

,end,Company_Name,netCashFlowEndOfPeriod,cfo
0,2020-03-31,Tesla,8547000000,-440000000
1,2020-06-30,Tesla,9106000000,964000000
2,2020-09-30,Tesla,15003000000,2400000000
3,2020-12-31,Tesla,19901000000,3019000000
4,2021-03-31,Tesla,17723000000,1641000000


In [25]:
# To get the historical data of Tesla's equity
# and transfer into DataFrame and drop any rows with missing data
totalSHequity_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['StockholdersEquity']['units']['USD'])).\
               dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
totalSHequity_tesla_df = totalSHequity_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'totalShareholderEquity'})


In [26]:
# Add a new column to tesla financial dataframe
tsla_financial_df['totalShareholderEquity'] = totalSHequity_tesla_df['totalShareholderEquity']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity
9,2022-06-30,Tesla,18887000000,2351000000,36376000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000


In [27]:
# To get the historical data of Tesla's short-term debt
# and transfer into DataFrame and drop any rows with missing data

shortTermDebt_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['DebtCurrent']['units']['USD'])).\
               dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
shortTermDebt_tesla_df = shortTermDebt_tesla_df[-14:].reset_index(drop=True)
shortTermDebt_tesla_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,1057000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I
10,2022-09-30,979000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2022Q3I
11,2022-12-31,1016000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q4I
12,2023-03-31,935000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1I
13,2023-06-30,1015000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2I


In [28]:
# To get the historical data of Tesla's current finance lease 
# and transfer into DataFrame and drop any rows with missing data

currentFinanceLease_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['FinanceLeaseLiabilityCurrent']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentFinanceLease_tesla_df = currentFinanceLease_tesla_df[-14:].reset_index(drop=True)
currentFinanceLease_tesla_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,475000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I
10,2022-09-30,478000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2022Q3I
11,2022-12-31,486000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q4I
12,2023-03-31,469000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1I
13,2023-06-30,444000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2I


In [29]:
# Calculate the total short-term debt
total_short_term_debt = shortTermDebt_tesla_df['val'] + currentFinanceLease_tesla_df['val']

# Add a new column to tesla financial dataframe
tsla_financial_df['totalShortTermDebt'] = total_short_term_debt
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000


In [30]:
# To get the historical data of Tesla's non-current finance lease
# and transfer into DataFrame and drop any rows with missing data
noncurrentFinanceLease_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['FinanceLeaseLiabilityNoncurrent']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
noncurrentFinanceLease_tesla_df = noncurrentFinanceLease_tesla_df[-14:].reset_index(drop=True)
noncurrentFinanceLease_tesla_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,803000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I
10,2022-09-30,678000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2022Q3I
11,2022-12-31,568000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q4I
12,2023-03-31,455000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1I
13,2023-06-30,368000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2I


In [31]:
# To get the historical data of Tesla's long-term debt and transfer into DataFrame
# and transfer into DataFrame and drop any rows with missing data
longTermDebt_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['LongTermDebt']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
longTermDebt_tesla_df = longTermDebt_tesla_df[-14:].reset_index(drop=True)
longTermDebt_tesla_df.tail()

,end,val,accn,fy,fp,form,filed,frame
9,2022-06-30,2095000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2022Q2I
10,2022-09-30,1418000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2022Q3I
11,2022-12-31,1029000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q4I
12,2023-03-31,817000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1I
13,2023-06-30,504000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2I


In [32]:
# Calculate the total long-term debt
total_long_term_debt = longTermDebt_tesla_df['val'] + noncurrentFinanceLease_tesla_df['val']

# Add a new column to tesla financial dataframe
tsla_financial_df['totalLongTermDebt'] = total_long_term_debt
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000


In [33]:
# To get the historical data of Tesla's short-term investment
# and transfer into DataFrame and drop any rows with missing data
shortTermInvest_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['ShortTermInvestments']['units']['USD'])).dropna()
# Review the DataFrame
shortTermInvest_tesla_df

,end,val,accn,fy,fp,form,filed,frame
0,2021-12-31,131000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021Q4I
3,2022-12-31,5932000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q4I
4,2023-03-31,6354000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1I
5,2023-06-30,7779000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2I


In [34]:
# To get the historical data of Tesla's cash and cash equivalents
# and transfer into DataFrame and drop any rows with missing data
cash_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['CashAndCashEquivalentsAtCarryingValue']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
cash_tesla_df = cash_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'cashAndCashEquiva'})

In [35]:
# Add a new column to tesla financial dataframe
tsla_financial_df['cashAndCashEquiva'] = cash_tesla_df['cashAndCashEquiva']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000


In [36]:
# To get the historical data of Tesla's current liabilities
# and transfer into DataFrame and drop any rows with missing data
currentLiabilities_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentLiabilities_tesla_df = currentLiabilities_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'CL'})


In [37]:
# Add a new column to tesla financial dataframe
tsla_financial_df['CL'] = currentLiabilities_tesla_df['CL']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000


In [38]:
# To get the historical data of Tesla's current assets
# and transfer into DataFrame and drop any rows with missing data

currentAssets_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['AssetsCurrent']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
currentAssets_tesla_df = currentAssets_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'CA'})


In [39]:
# Add a new column to tesla financial dataframe
tsla_financial_df['CA'] = currentAssets_tesla_df['CA']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000,35990000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000,40917000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000,42997000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000,43875000000


In [40]:
# To get the historical data of Tesla's inventory
# and transfer into DataFrame and drop any rows with missing data

inventory_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['InventoryNet']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
inventory_tesla_df = inventory_tesla_df[-14:].reset_index(drop=True).rename(columns={'val':'inventory'})


In [41]:
# Add a new column to tesla financial dataframe
tsla_financial_df['inventory'] = inventory_tesla_df['inventory']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000,8108000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000,35990000000,10327000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000,40917000000,12839000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000,42997000000,14375000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000,43875000000,14356000000


In [42]:
# To get the historical data of Tesla's total cost of revenue
# and transfer into DataFrame and drop any rows with missing data

totalCostOfRevenue_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['CostOfRevenue']['units']['USD'])).dropna()
# Review the DataFrame
totalCostOfRevenue_tesla_df.tail()

,start,end,val,accn,fy,fp,form,filed,frame
156,2022-04-01,2022-06-30,12700000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2
158,2022-07-01,2022-09-30,16072000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2022Q3
159,2022-01-01,2022-12-31,60609000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2022
160,2023-01-01,2023-03-31,18818000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2023Q1
162,2023-04-01,2023-06-30,20394000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2023Q2


In [43]:
# To get the historical data of Tesla's Income from Operations
# and transfer into DataFrame and drop any rows with missing data
operatingIncome_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['OperatingIncomeLoss']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
operatingIncome_tesla_df = operatingIncome_tesla_df[-14:].reset_index(drop=True)
operatingIncome_tesla_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,283000000,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,CY2020Q1
1,2020-04-01,2020-06-30,327000000,0000950170-21-000524,2021,Q2,10-Q,2021-07-27,CY2020Q2
2,2020-07-01,2020-09-30,809000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,CY2020Q3
3,2020-01-01,2020-12-31,1994000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2020
4,2021-01-01,2021-03-31,594000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2021Q1
5,2021-04-01,2021-06-30,1312000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q2
6,2021-07-01,2021-09-30,2004000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2021Q3
7,2021-01-01,2021-12-31,6523000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021
8,2022-01-01,2022-03-31,3603000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2022Q1
9,2022-04-01,2022-06-30,2464000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2


In [44]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the differences
new_operatingIncome = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = operatingIncome_tesla_df.at[i, 'val'] - operatingIncome_tesla_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = operatingIncome_tesla_df.at[i, 'val']
    new_operatingIncome.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
new_operatingIncome.append(operatingIncome_tesla_df.at[12, 'val'])
new_operatingIncome.append(operatingIncome_tesla_df.at[13, 'val'])

In [45]:
# Add a new column to tesla financial dataframe
tsla_financial_df['incomeFromOperations'] = new_operatingIncome
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000,8108000000,2464000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000,35990000000,10327000000,3688000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000,40917000000,12839000000,3901000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000,42997000000,14375000000,2664000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000,43875000000,14356000000,2399000000


In [46]:
# To get the historical data of Tesla's Total Revenue and transfer into DataFrame and drop any rows with missing data
# Drop duplicated rows per column reporting end date "end"
totalRevenue_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['Revenues']['units']['USD'])).dropna().drop_duplicates(subset=['end'])

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
totalRevenue_tesla_df = totalRevenue_tesla_df[-14:].reset_index(drop=True)
totalRevenue_tesla_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,5985000000,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,CY2020Q1
1,2020-04-01,2020-06-30,6036000000,0000950170-21-000524,2021,Q2,10-Q,2021-07-27,CY2020Q2
2,2020-07-01,2020-09-30,8771000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,CY2020Q3
3,2020-01-01,2020-12-31,31536000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2020
4,2021-01-01,2021-03-31,10389000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2021Q1
5,2021-04-01,2021-06-30,11958000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q2
6,2021-07-01,2021-09-30,13757000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2021Q3
7,2021-01-01,2021-12-31,53823000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021
8,2022-01-01,2022-03-31,18756000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2022Q1
9,2022-04-01,2022-06-30,16934000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2


In [47]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the differences
new_total_rev = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = totalRevenue_tesla_df.at[i, 'val'] - totalRevenue_tesla_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = totalRevenue_tesla_df.at[i, 'val']
    new_total_rev.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
new_total_rev.append(totalRevenue_tesla_df.at[12, 'val'])
new_total_rev.append(totalRevenue_tesla_df.at[13, 'val'])

In [48]:
# Add a new column to tesla financial dataframe
tsla_financial_df['totalRevenue'] = new_total_rev
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000,8108000000,2464000000,16934000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000,35990000000,10327000000,3688000000,21454000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000,40917000000,12839000000,3901000000,24318000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000,42997000000,14375000000,2664000000,23329000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000,43875000000,14356000000,2399000000,24927000000


In [49]:
# To get the historical data of Tesla's Net Income Attributable to common stockholders
# and transfer into DataFrame and drop any rows with missing data
netIncomeAttrCommonStock_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['NetIncomeLossAvailableToCommonStockholdersBasic']['units']['USD'])).dropna()

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
# NOTE: 2020 Q1,Q2 and Q4 data is missing and will be added back next from previous finanial report
# Their related reporting will be matched once they combined to new tesla dataframe
netIncomeAttrCommonStock_tesla_df = netIncomeAttrCommonStock_tesla_df[-14:].reset_index(drop=True)
netIncomeAttrCommonStock_tesla_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2019-07-01,2019-09-30,143000000,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,CY2019Q3
1,2019-01-01,2019-12-31,-870000000,0000950170-22-000796,2021,FY,10-K,2022-02-07,CY2019
2,2020-07-01,2020-09-30,300000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,CY2020Q3
3,2020-01-01,2020-12-31,690000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2020
4,2021-01-01,2021-03-31,438000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2021Q1
5,2021-04-01,2021-06-30,1142000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q2
6,2021-07-01,2021-09-30,1618000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2021Q3
7,2021-01-01,2021-12-31,5524000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021
8,2022-01-01,2022-03-31,3313000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2022Q1
9,2022-04-01,2022-06-30,2256000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2


In [50]:
# Adjust some rows data to reflect quarterly data

# Add back 2020 Q1,Q2 and Q4 missing data
netIncomeAttrCommonStock_tesla_df.at[0, "val"] = 16000000
netIncomeAttrCommonStock_tesla_df.at[1, "val"] = 104000000
netIncomeAttrCommonStock_tesla_df.at[3, "val"] = 270000000

# 2022 Q4 Data
Q2_2022_netincomeAttrCommonStock = netIncomeAttrCommonStock_tesla_df.iloc[11]['val'] - netIncomeAttrCommonStock_tesla_df.iloc[8:11]['val'].sum()
netIncomeAttrCommonStock_tesla_df.at[11, "val"] = Q2_2022_netincomeAttrCommonStock

# 2021 Q4 Data
Q2_2021_netincomeAttrCommonStock = netIncomeAttrCommonStock_tesla_df.iloc[7]['val'] - netIncomeAttrCommonStock_tesla_df.iloc[4:7]['val'].sum()
netIncomeAttrCommonStock_tesla_df.at[7, "val"] = Q2_2021_netincomeAttrCommonStock

netIncomeAttrCommonStock_tesla_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2019-07-01,2019-09-30,16000000,0001564590-20-047486,2020,Q3,10-Q,2020-10-26,CY2019Q3
1,2019-01-01,2019-12-31,104000000,0000950170-22-000796,2021,FY,10-K,2022-02-07,CY2019
2,2020-07-01,2020-09-30,300000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,CY2020Q3
3,2020-01-01,2020-12-31,270000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2020
4,2021-01-01,2021-03-31,438000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2021Q1
5,2021-04-01,2021-06-30,1142000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q2
6,2021-07-01,2021-09-30,1618000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2021Q3
7,2021-01-01,2021-12-31,2326000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021
8,2022-01-01,2022-03-31,3313000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2022Q1
9,2022-04-01,2022-06-30,2256000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2


In [51]:
# Add a new column to tesla financial dataframe
tsla_financial_df['netIncomeAttrCommonStock'] = netIncomeAttrCommonStock_tesla_df['val']
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue,netIncomeAttrCommonStock
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000,8108000000,2464000000,16934000000,2256000000
10,2022-09-30,Tesla,20149000000,5100000000,39851000000,1457000000,2096000000,19532000000,24611000000,35990000000,10327000000,3688000000,21454000000,3292000000
11,2022-12-31,Tesla,16924000000,3278000000,44704000000,1502000000,1597000000,16253000000,26709000000,40917000000,12839000000,3901000000,24318000000,3722000000
12,2023-03-31,Tesla,16770000000,2513000000,48054000000,1404000000,1272000000,16048000000,27436000000,42997000000,14375000000,2664000000,23329000000,2518000000
13,2023-06-30,Tesla,15879000000,3065000000,51130000000,1459000000,872000000,15296000000,27592000000,43875000000,14356000000,2399000000,24927000000,2703000000


In [52]:
# To get the historical data of Tesla's Gross profit and transfer into DataFrame and transfer into DataFrame and drop any rows with missing data
# Drop any duplicated rows with missing data

grossProfit_tesla_df = pd.DataFrame.from_dict((
               companyFacts_tesla.json()['facts']['us-gaap']['GrossProfit']['units']['USD'])).dropna().drop_duplicates(subset=['end'])

# Only keep the last 14 rows (from 2020 Q1 - 2023 Q2) and reset index
grossProfit_tesla_df = grossProfit_tesla_df[-14:].reset_index(drop=True)
grossProfit_tesla_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2020-01-01,2020-03-31,1234000000,0000950170-21-000046,2021,Q1,10-Q,2021-04-28,CY2020Q1
1,2020-04-01,2020-06-30,1267000000,0000950170-21-000524,2021,Q2,10-Q,2021-07-27,CY2020Q2
2,2020-07-01,2020-09-30,2063000000,0000950170-21-002253,2021,Q3,10-Q,2021-10-25,CY2020Q3
3,2020-01-01,2020-12-31,6630000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2020
4,2021-01-01,2021-03-31,2215000000,0000950170-22-006034,2022,Q1,10-Q,2022-04-25,CY2021Q1
5,2021-04-01,2021-06-30,2884000000,0000950170-22-012936,2022,Q2,10-Q,2022-07-25,CY2021Q2
6,2021-07-01,2021-09-30,3660000000,0000950170-22-019867,2022,Q3,10-Q,2022-10-24,CY2021Q3
7,2021-01-01,2021-12-31,13606000000,0000950170-23-001409,2022,FY,10-K,2023-01-31,CY2021
8,2022-01-01,2022-03-31,5460000000,0000950170-23-013890,2023,Q1,10-Q,2023-04-24,CY2022Q1
9,2022-04-01,2022-06-30,4234000000,0000950170-23-033872,2023,Q2,10-Q,2023-07-24,CY2022Q2


In [53]:
# Adjust some rows data to reflect quarterly data
# Create an empty list to store the differences
new_gross = []

# Calculate the fourth quarter data only (2020 -2022)
for i in range(12):
    if i % 4 == 3:
        difference = grossProfit_tesla_df.at[i, 'val'] - grossProfit_tesla_df.loc[i-3:i-1, 'val'].sum()
    else:
        difference = grossProfit_tesla_df.at[i, 'val']
    new_gross.append(difference)

# Add the 2023 Q1 and Q2 data to the new list
new_gross.append(grossProfit_tesla_df.at[12, 'val'])
new_gross.append(grossProfit_tesla_df.at[13, 'val'])

In [56]:
# Add a new column to tesla financial dataframe
tsla_financial_df['grossProfit'] = new_gross
tsla_financial_df.tail()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue,netIncomeAttrCommonStock,grossProfit
0,2020-03-31,Tesla,8547000000,-440000000,9173000000,3217000000,10666000000,8080000000,11986000000,14893000000,4494000000,283000000,5985000000,16000000,1234000000
1,2020-06-30,Tesla,9106000000,964000000,9855000000,3679000000,10416000000,8615000000,12270000000,15336000000,4018000000,327000000,6036000000,104000000,1267000000
2,2020-09-30,Tesla,15003000000,2400000000,16031000000,3126000000,10559000000,14531000000,13302000000,21744000000,4218000000,809000000,8771000000,300000000,2063000000
3,2020-12-31,Tesla,19901000000,3019000000,22225000000,2132000000,9556000000,19384000000,14248000000,26717000000,4101000000,575000000,10744000000,270000000,2066000000
4,2021-03-31,Tesla,17723000000,1641000000,23017000000,1819000000,9053000000,17141000000,14877000000,24705000000,4132000000,594000000,10389000000,438000000,2215000000
5,2021-06-30,Tesla,16825000000,2124000000,24804000000,1530000000,7871000000,16229000000,16371000000,24693000000,4733000000,1312000000,11958000000,1142000000,2884000000
6,2021-09-30,Tesla,16694000000,3147000000,27053000000,1716000000,6438000000,16065000000,18051000000,25002000000,5199000000,2004000000,13757000000,1618000000,3660000000
7,2021-12-31,Tesla,18144000000,4585000000,30189000000,1589000000,5245000000,17576000000,19705000000,27100000000,5757000000,2613000000,17719000000,2326000000,4847000000
8,2022-03-31,Tesla,18040000000,3995000000,34085000000,1659000000,3153000000,17505000000,21455000000,29050000000,6691000000,3603000000,18756000000,3313000000,5460000000
9,2022-06-30,Tesla,18887000000,2351000000,36376000000,1532000000,2898000000,18324000000,21821000000,31222000000,8108000000,2464000000,16934000000,2256000000,4234000000


In [55]:
# Write the DataFrame to a excel file
tsla_financial_df.to_excel("Outputs/tsla_financial_measures.xlsx", index=False)

### Combine Tesla and Ford Motor's DataFrame

In [167]:
# Combine tesla financial dataframe and ford finacial dataframe
result_df = pd.concat([tsla_financial_df, ford_financial_df], ignore_index=True)
result_df.head()

,end,Company_Name,netCashFlowEndOfPeriod,cfo,totalShareholderEquity,totalShortTermDebt,totalLongTermDebt,cashAndCashEquiva,CL,CA,inventory,incomeFromOperations,totalRevenue,netIncomeAttrCommonStock,grossProfit
0,2020-03-31,Tesla,8547000000,-440000000,9173000000,3217000000,10666000000,8080000000,11986000000,14893000000,4494000000,283000000,5985000000,16000000,1234000000
1,2020-06-30,Tesla,9106000000,964000000,9855000000,3679000000,10416000000,8615000000,12270000000,15336000000,4018000000,327000000,6036000000,104000000,1267000000
2,2020-09-30,Tesla,15003000000,2400000000,16031000000,3126000000,10559000000,14531000000,13302000000,21744000000,4218000000,809000000,8771000000,300000000,2063000000
3,2020-12-31,Tesla,19901000000,3019000000,22225000000,2132000000,9556000000,19384000000,14248000000,26717000000,4101000000,575000000,10744000000,270000000,2066000000
4,2021-03-31,Tesla,17723000000,1641000000,23017000000,1819000000,9053000000,17141000000,14877000000,24705000000,4132000000,594000000,10389000000,438000000,2215000000


In [168]:
# Write the DataFrame to a excel file
result_df.to_excel("Outputs/combined_financial_measures.xlsx", index=False)